<a href="https://colab.research.google.com/github/rtegao/M5Forecasting/blob/master/ExploratoryDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [0]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Databases

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Contains information about the dates the products are sold.
df_calendar = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/calendar.csv')

In [0]:
# Contains information about the price of the products sold per store and date.
df_price = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/sell_prices.csv')

In [0]:
#Contains the historical daily unit sales data per product and store.
df_sales = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/sales_train_validation.csv')

# Visualizing DataBases

In [6]:
df_calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [7]:
df_price

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [8]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


# Understanding a little bit more about df_sales

In [9]:
fig = px.histogram(df_sales, x="state_id",title='QTY. of stores by state')
fig.show()

In [10]:
fig = px.histogram(df_sales, x="cat_id",title='QTY. of each category')
fig.show()

In [11]:
fig = px.histogram(df_sales, x="state_id", color="cat_id",title='QTY. of category by state')
fig.show()

In [12]:
# sales quantity through time by state

states = df_sales.groupby('state_id').sum().reset_index()

fig = go.Figure()
for i in range(len(states)):
  fig.add_trace(go.Scatter(x=list(range(states.shape[1])), y=states.iloc[i,1:],
                      mode='lines+markers',
                      name=states.loc[i,'state_id']))
fig.update_layout(title='Sales Quantity Through Time by State',
                   xaxis_title='Days',
                   yaxis_title='Quantity')
fig.show()

In [13]:
# Sales Quantity Through Time by Category

categories = df_sales.groupby('cat_id').sum().reset_index()

fig = go.Figure()
for i in range(len(categories)):
  fig.add_trace(go.Scatter(x=list(range(categories.shape[1])), y=categories.iloc[i,1:],
                      mode='lines+markers',
                      name=categories.loc[i,'cat_id']))
fig.update_layout(title='Sales Quantity Through Time by Category',
                   xaxis_title='Days',
                   yaxis_title='Quantity')
fig.show()

## Analyzing the graphs above it's possible to see that in 5 days the sales goes next to zero. Using the others DataFrames i will try to understand the reason.

### The days below are the ones with bad sales. There are just one event in all of them, unfortunately, this made our analysis inconclusive.


In [14]:
df_calendar[(df_calendar['d']=='d_330')|(df_calendar['d']=='d_696')|(df_calendar['d']=='d_1061')|(df_calendar['d']=='d_1426')|(df_calendar['d']=='d_1791')]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
329,2011-12-24,11148,Saturday,1,12,2011,d_330,NaN,NaN,NaN,NaN,0,0,0
695,2012-12-24,11248,Monday,3,12,2012,d_696,NaN,NaN,NaN,NaN,0,0,0
1060,2013-12-24,11348,Tuesday,4,12,2013,d_1061,NaN,NaN,NaN,NaN,0,0,0
1425,2014-12-24,11447,Wednesday,5,12,2014,d_1426,Chanukah End,Religious,NaN,NaN,0,0,0
1790,2015-12-24,11547,Thursday,6,12,2015,d_1791,NaN,NaN,NaN,NaN,0,0,0


# Lets choose one product by category, by state and plot they behavior through time.

In [0]:
hobbies = df_sales[(df_sales['dept_id'].str.contains('HOBBIES'))].groupby('state_id', as_index=False)
foods = df_sales[(df_sales['dept_id'].str.contains('FOOD'))].groupby('state_id', as_index=False)
households = df_sales[(df_sales['dept_id'].str.contains('HOUSEHOLD'))].groupby('state_id', as_index=False)

## Analysing the graphs below, its possibel to see that the same product have a similar behavior in differents states

In [16]:
fig = go.Figure()
for i in range(hobbies.first().shape[0]):
  fig.add_trace(go.Scatter(x=list(range(hobbies.first().loc[i,'d_1':].shape[0])),y = hobbies.first().loc[i,'d_1':], mode ='lines+markers',name = hobbies.first().loc[i,'state_id']))
  fig.update_layout(title = 'Hobbies Product QTY. Sales Through Time',
                    xaxis_title = 'Days',
                    yaxis_title = 'Quantity') 
fig.show()

In [17]:
fig = go.Figure()
for i in range(foods.first().shape[0]):
  fig.add_trace(go.Scatter(x=list(range(foods.first().loc[i,'d_1':].shape[0])),y = foods.first().loc[i,'d_1':], mode ='lines+markers',name = foods.first().loc[i,'state_id']))
  fig.update_layout(title = 'Food Product QTY. Sales Through Time',
                  xaxis_title = 'Days',
                  yaxis_title = 'Quantity') 
fig.show()

In [18]:
fig = go.Figure()
for i in range(households.first().shape[0]):
  fig.add_trace(go.Scatter(x=list(range(households.first().loc[i,'d_1':].shape[0])),y = households.first().loc[i,'d_1':], mode ='lines+markers',name = households.first().loc[i,'state_id']))
  fig.update_layout(title = 'House Holds Product QTY. Sales Through Time',
                    xaxis_title = 'Days',
                    yaxis_title = 'Quantity') 
fig.show()

# Lets take 4 differents products from the same category and store:

## Graph interpretation:
- ## Some differents products from the same catecory may have the same selling pattern.


In [19]:
hobbies = df_sales[(df_sales['dept_id'].str.contains('HOBBIES')) & (df_sales['state_id'].str.contains('CA'))].loc[0:4,:]
fig = go.Figure()
for i in range(hobbies.shape[0]):
  fig.add_trace(go.Scatter(x=list(range(hobbies.loc[i,'d_1':].shape[0])),y = hobbies.loc[i,'d_1':], mode ='lines+markers',name =hobbies.loc[i,'item_id']))
    
fig.update_layout(title = 'Hobbies Products QTY. Sales Through Time',
                    xaxis_title = 'Days',
                    yaxis_title = 'Quantity') 
fig.show()

In [20]:
foods = df_sales[(df_sales['dept_id'].str.contains('FOODS')) & (df_sales['state_id'].str.contains('CA'))].reset_index().loc[0:4,:]
fig = go.Figure()
for i in range(foods.shape[0]):
  fig.add_trace(go.Scatter(x=list(range(foods.loc[i,'d_1':].shape[0])),y = foods.loc[i,'d_1':], mode ='lines+markers',name =foods.loc[i,'item_id']))
    
fig.update_layout(title = 'Food Products QTY. Sales Through Time',
                    xaxis_title = 'Days',
                    yaxis_title = 'Quantity') 
fig.show()

In [21]:
households = df_sales[(df_sales['dept_id'].str.contains('HOUSE')) & (df_sales['state_id'].str.contains('CA'))].reset_index().loc[0:4,:]
fig = go.Figure()
for i in range(households.shape[0]):
  fig.add_trace(go.Scatter(x=list(range(households.loc[i,'d_1':].shape[0])),y = households.loc[i,'d_1':], mode ='lines+markers',name =households.loc[i,'item_id']))
    
fig.update_layout(title = 'Households Products QTY. Sales Through Time',
                    xaxis_title = 'Days',
                    yaxis_title = 'Quantity') 
fig.show()

# Resume:
- # df_sales its the DataFrame used for forecasting
- # df_calendar and df_sell_prices can be use for add new features in the system
- # A hypothesis is that the same forecasting system can be generalized for one product in differents states.
- # Another hysthesis is that some products from the same category can have the same selling pattern